In [64]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from glob import glob
import torchvision


In [65]:
class customdataset(Dataset): 
    def __init__(self, data_dir, transform= None): 
        #이닛 해주고
        self.data_dir= data_dir 
        self.transform= transform
        #이미지들 경로명 담을 리스트
        self.image_paths= []
        self.labels= []

        #이제 여기서 데이터를 결정해줘야해. 
        #경로에서 'NORMAL' 폴더에 있는 하위 파일들을 모두 리스트로 만듦
        normal_dir = os.path.join(data_dir, 'NORMAL')
        for img in glob(f"{normal_dir}/*"): 
            self.image_paths.append(img)
            self.labels.append(0)

        pneumonia_dir = os.path.join(data_dir, 'NORMAL')
        for img in glob(f"{pneumonia_dir}/*"): 
            self.image_paths.append(img)
            self.labels.append(1)
            
    def __len__(self): 
        #len이건 필수래 나중에 DataLoader 하려면 
        #몇번 반복해서 받을건지를 결정하는 요소? 
        #이미지파일 갯수만큼 받아야하니까
        return len(self.image_paths)

    def __getitem__(self, idx): 
        #이것도 필수래 DataLoader 에서 이터레이터한 객체를 받아올 수 있다는데? 
        #그래서 for문같은 반복문 추가 안해줘도 DataLoader에서 알아서 반복 해준대 ㅇㅇ
        #이미지 경로를 받아서 실제 이미지를 열어주는 구간
        
        image= torchvision.io.read_image(self.image_paths[idx])
        label= self.labels[idx]

        if self.transform: 
            image= self.transform(image)

        return image, label


In [75]:
train_transform= transforms.Compose([ 
    #데이터 전처리 하기위해서 필수인 과정 
    #데이터 증강도 할 수 있음 +a인 느낌 
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1), 
    transforms.RandomHorizontalFlip(),      # 수평 뒤집기
    transforms.RandomRotation(10),          # 10도 이내로 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # 밝기 및 대비 조절
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # 회전 없이 이동 변환만 적용
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])

val_transform= transforms.Compose([
    #검증 데이터셋을 만들 땐 전처리만
    #데이터 증강은 필요없음. 
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])


In [76]:
full_dataset= customdataset(data_dir= 'chest_xray/train', transform= train_transform) 
train_size= int(0.8*len(full_dataset)) 
val_size= len(full_dataset) - train_size
train_dataset, val_dataset= random_split(full_dataset, [train_size, val_size]) 


In [77]:
print(full_dataset.transform, train_dataset.dataset.transform)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=1)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
) Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=1)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
)


In [78]:

# 검증 데이터는 데이터 증강을 적용하지 않기 때문에 변환을 따로 지정
val_dataset.dataset = customdataset(data_dir='chest_xray/train', transform=val_transform)


In [80]:
print(full_dataset.transform, train_dataset.dataset.transform, val_dataset.dataset.transform)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=1)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
) Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=1)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
) Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Grayscal

In [81]:
# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [33]:
class EnhancedCNN(mm.Module): 
    

17